In [1]:
##################################################################
################### last update 20240212 #########################
##################################################################


#importing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as plticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import sys
import scipy
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import math
from math import e
import pandas as pd
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib import collections  as mc


#definition mean value
def mean(v1,v2):
    return [meanlist(v1), meanlist(v2)]
#definition deviation
def std(v1,v2):
    return (stdlist(v1), stdlist(v2))

#mean value for list of indicator rankings
def meanlist(l2):
    mean=0
    l= [v for v in l2 if not (math.isinf(v) or math.isnan(v))]
    l1=np.zeros(len(l))
    for i in range(len(l)):
        l1[i]+=l[i]
        mean += l1[i]
    if len(l)==0:
        meanvalue=np.inf
    else:
        meanvalue= mean/len(l) 
    return meanvalue

def stdlist(l):
    stdl=0
    m=meanlist(l)
    for i in range(len(l)):
        stdl += (l[i]-m)**2
    return np.sqrt(stdl/len(l-1))

def meanlistreturn(meanlists, length_of_video):
    listreturn = np.zeros(length_of_video)
    for i in range(length_of_video):
        m=meanlist(meanlists[i]) 
        round_m=round(m)
        listreturn[i]+=round_m
    return np.array(listreturn)
#random sample with destrubution
def random_sample(grid_size,n_points,distribution,**params):
    def uniform():
        return np.array(np.random.randint(0,grid_size+2,n_points))
    
    def brownian(mu=grid_size/2,sig=np.sqrt(grid_size)):
        rand = np.zeros(n_points)
        rand[0] = mu
        for i in range(1,n_points):
            tmp = round(np.random.randn()*sig +rand[i-1])
            while (tmp < 1 or tmp > grid_size):
                tmp = round(np.random.randn()*sig +rand[i-1])
            rand[i]=tmp
        return np.array(rand)
    def gauss(mu=grid_size/2,sig=np.sqrt(grid_size)):
        rand = np.round(np.random.randn(n_points)*sig+mu).astype(int)
        i = np.where(np.logical_or(rand<1,rand>grid_size))[0]
        while len(i)>0:
            rand2 = np.round(np.random.randn(len(i))*sig+mu).astype(int)
            rand[i] = rand2
            i = np.where(np.logical_or(rand<1,rand>=grid_size))[0]
        return np.array(rand)
       
    if distribution=='uniform':
        return uniform()
    if distribution=='gauss':
        return gauss(**params)
    if distribution=='brownian':
        return brownian(**params)
#recombinations for intercorrelation
def rater(v, j):
    return [b[j]for b in v]
def ratermeanvaluelist(v):
    l=np.zeros(len(v[1]))
    for i in range(len(v[1])):
        l[i] += round(meanlist(rater(v,i)))
    return l
#desity of points to deviationcurve
def density(v1,v2):
    counter=0
    m = mean(v1,v2)
    dm = std(v1,v2)
#angle for deviationscurve
    phi = np.linspace(0,2*np.pi,100)
#defination of deviationcurve
    cx = m[0]+dm[0]*np.cos(phi)
    cy = m[1]+dm[1]*np.sin(phi)
    for i in range(len(v1)):
        distance_to_ev=((m[0]-v1[i])**2+(m[1]-v2[i])**2)**0.5
        distance_to_curve=np.zeros(100)
        distance_ev_to_curve=np.zeros(100)
        for j in range(100):
            distance_to_curve[j] += ((cx[j]-v1[i])**2+(cy[j]-v2[i])**2)**0.5
        min_distance_to_curve=meanlist(distance_to_curve)
        for k in range(100):
            distance_ev_to_curve[k] += ((cx[k]-m[0])**2+(cy[k]-m[1])**2)**0.5
        min_distance_ev_to_curve=meanlist(distance_ev_to_curve)
        if min_distance_ev_to_curve>distance_to_ev:
            counter += 1
        else:
            counter += 0
    return counter/len(v1)
#traveltendency for 2 dimensions
def traveltendency(v1, v2):
    tt=0
    td1=np.zeros(len(v1)-1)
    td2=np.zeros(len(v2)-1)
    for i in range(len(v1)-1):
        td1[i] += (v1[i]-v1[i+1])
        td2[i] += (v2[i]-v2[i+1])
        if td1[i]<0:
            tt+=1
        if td1[i]==0:
            tt+=0
        if td1[i]>0:
            tt+=-1
        if td2[i]<0:
            tt+=1
        if td2[i]==0:
            tt+=0
        if td2[i]>0:
            tt+=-1
    return tt/(2*(len(v1)-1))
#traveldirection for 2 dimensions
def traveldistance(v1, v2):
    td=0
    for i in range(len(v1)-1):
        td += ((v1[i]-v1[i+1])**2+(v2[i]-v2[i+1])**2)**0.5
    return td/(len(v1)-1)
#linear function for curve_fit
def lin(x, a, b):
    x=np.array(x)
    return(a*x+b)
def quadfkt(x, a, b, c):
    x=np.array(x)
    return(a*x**2+b*x+c)


def timeconst(v):
    number = np.zeros(len(v))+1
    for i in range(len(v)):
        for j in range(i):
            if v[i] == v[j]:
               number[i] += 10   
    return number*50

def grid_chessboard(x_limit, y_limit):
    background = plt.Rectangle((0,0), x_limit, y_limit, fc='lightgray', zorder=0)
    plt.gca().add_patch(background)
    for x in range(x_limit):
        for y in range(y_limit):
            rectangle1 = plt.Rectangle((2*x+0.5,2*y+0.5), 1, 1, fc='darkgrey', zorder=1)
            rectangle2 = plt.Rectangle((2*x+1.5,2*y+1.5), 1, 1, fc='darkgrey', zorder=1)
            plt.gca().add_patch(rectangle1)
            plt.gca().add_patch(rectangle2)
            
def time(x, y):
    size=[100]
    for i in range(len(x)-1):
        if x[i]==x[i+1] and y[i]==y[i+1]:
            size.append((size[i]+200))
        else:
            size.append(100)
    return size

def relocating(x, y):
    x_new=np.zeros(len(x))
    y_new=np.zeros(len(y))
    size=time(x,y)
    x_new[0]=(x[0]+0.5*((0+1)/(0+2))**0.45*np.cos((0+1)/(0+2)*8*np.pi))
    y_new[0]=(y[0]+0.5*((0+1)/(0+2))**0.45*np.sin((0+1)/(0+2)*8*np.pi))
    for j in range(len(x)-1):
        i=j+1
        if x[i]==x[i-1] and y[i]==y[i-1]:
            x_new[i] =+ x_new[i-1]
            y_new[i] =+ y_new[i-1]
        else:
            x_new[i] =+ x[i]+0.45*((i+1)/(len(x)+2))**0.5*np.cos((i+1)/(len(x)+2)*8*np.pi)
            y_new[i] =+ y[i]+0.45*((i+1)/(len(x)+2))**0.5*np.sin((i+1)/(len(x)+2)*8*np.pi)
        if x_new[i]==0 and y_new[i]==0:
            x_new[i] =+ x[i]+0.45*((i+1)/(len(x)+2))**0.5*np.cos((i+1)/(len(x)+2)*8*np.pi)
            y_new[i] =+ y[i]+0.45*((i+1)/(len(x)+2))**0.5*np.sin((i+1)/(len(x)+2)*8*np.pi)
    return x_new, y_new


def ratings_mean_and_error(all_ratings, number_of_raters, number_of_ratings):
    mean_value_each_rating=np.zeros(int(number_of_ratings))
    error_each_rating=np.zeros(int(number_of_ratings))
    for i in range(int(number_of_ratings)):
        rating_of_each_rater=np.zeros(int(number_of_raters))
        for j in range(int(number_of_raters)):
            k=i+j*number_of_ratings
            rating_of_each_rater[j]=+(all_ratings[k])
        mean_value_each_rating[i]=+(round(meanlist(rating_of_each_rater)))
        error_each_rating[i]=+(stdlist(rating_of_each_rater)/np.sqrt(number_of_raters))
    return mean_value_each_rating, error_each_rating

def ratings_mean_not_rounded(all_ratings, number_of_raters, number_of_ratings):
    mean_value_each_rating=np.zeros(int(number_of_ratings))
    for i in range(int(number_of_ratings)):
        rating_of_each_rater=np.zeros(int(number_of_raters))
        for j in range(int(number_of_raters)):
            k=i+j*number_of_ratings
            rating_of_each_rater[j]=+(all_ratings[k])
        mean_value_each_rating[i]=+(meanlist(rating_of_each_rater))
    return mean_value_each_rating

In [ ]:
def plot_2D_ASSG(path_data1, path_data2, path_fig, title, name_indicator_1, name_indicator_2):
    x_data_not_arranged=np.loadtxt(path_data1, unpack=True)
    x_data=np.array(x_data_not_arranged).T
    y_data_not_arranged=np.loadtxt(path_data2, unpack=True)
    y_data=np.array(y_data_not_arranged).T
    all_ratings_x=np.zeros(len(x_data_not_arranged[0])*len(x_data[1]))
    for j in range(len(x_data_not_arranged[0])):#slope of rater
        for k in range(len(x_data[0])):
            position_rating_x=k+j*len(x_data[j])
            all_ratings_x[position_rating_x]=+(x_data[j][k])
    all_ratings_y=np.zeros(len(y_data_not_arranged[1])*len(y_data[1]))
    for j in range(len(y_data_not_arranged[0])):#slope of rater
        for k in range(len(y_data[0])):
            position_rating_y=k+j*len(y_data[j])
            all_ratings_y[position_rating_y]=+(y_data[j][k])
    x, x_err = ratings_mean_and_error(all_ratings_x, len(x_data_not_arranged[1]), len(x_data[1]))
    y, y_err = ratings_mean_and_error(all_ratings_y, len(y_data_not_arranged[1]), len(y_data[1]))
    size=time(x,y)
    x_reloc, y_reloc = relocating(x,y)

###--------------------definations for the values---------------------------
#shorting of names for meanvalue and deviation
    m = [meanlist(x), meanlist(y)]
    dm = std(x,y)
#angle for deviation ellipse
    phi = np.linspace(0,2*np.pi,100)
#defination of deviation ellipse
    cx = m[0]+dm[0]*np.cos(phi)
    cy = m[1]+dm[1]*np.sin(phi)
#middle SD
    MSD = 0.5*(dm[0]+dm[1]) 
#SD difference
    SDD = np.absolute(dm[0]-dm[1])
#fitting
#linear
    popt, pcov = curve_fit(lin,x,y, bounds=([-5,-25], [5, 25]))
    xfit=np.linspace(0, 6)
    yfit=popt[0]*xfit+popt[1]
#xconst
    poptx, pcovx = curve_fit(xconst,x,y)
    xfit_xconst=np.linspace(0, 6)
    yfit_xconst=poptx[0]+0*xfit_xconst
#yconst
    popty, pcovy = curve_fit(xconst,y,x)
    yfit_yconst=np.linspace(0, 6)
    xfit_yconst=popty[0]+0*yfit_yconst
#inner point density
    dens=density(x,y) 
###---------------------------Plots-----------------------------------------
#size
    fig, ax = plt.subplots(figsize=(12,12))
#add grid
    grid_chessboard(5+1,5+1)
#points of ASSG-plot
    df = pd.DataFrame({'z': (np.linspace(0, len(x), len(x)))})
    dg = pd.DataFrame({'z': (np.linspace(len(x), 0, len(x)))})
    scatter_label= ax.scatter(x_reloc[0],y_reloc[0],s=size[0], c='none',label='td=%.2f, tt=%.2f' 
                                        %(traveldistance(x,y),traveltendency(x,y)), zorder=3)
    scatter= ax.scatter(x_reloc,y_reloc,s=size, c=df.z, cmap='Blues', zorder=4)
    scatter= ax.scatter(x_reloc,y_reloc,s=np.array(size)*1.2+100, c=dg.z, cmap='Blues', zorder=3)
#color errorbars
    #plt.errorbar(x_reloc, y_reloc, y_err, x_err, fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    errorbar_color = (np.linspace(0, len(x), len(x)))
    cmap = cm.Blues
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(x_reloc, y_reloc, x_err, y_err, fmt='none', ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
    
    errorbar_color_inside = (np.linspace(len(x), 0, len(x)))
    cmap = cm.Blues
    norm = Normalize(vmin=errorbar_color_inside.min(), vmax=errorbar_color_inside.max())
    plt.errorbar(x_reloc, y_reloc, x_err, y_err, fmt='none', ecolor=cmap(norm(errorbar_color_inside)),
                 elinewidth=3, zorder=10)
    
    plt.errorbar(x_reloc[-1], y_reloc[-1], x_err[-1], y_err[-1], fmt='none',
                 ecolor=cmap(norm(errorbar_color))[-1],
                 label='$\overline{\Delta x}$=%.2f $\pm$ %.2f, \n$\overline{\Delta y}$=%.2f $\pm$ %.2f'
                 %(meanlist(x_err), stdlist(x_err), meanlist(y_err), stdlist(y_err)), zorder=1)
    
    points=np.array([x_reloc, y_reloc]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(x), len(x)))
    
    
    
    
    (ax.set_xlabel('%s, $SD_x$=%s, $td_x$=%.2f, $tt_x$=%.2f'
                   %(name_indicator_1, round(dm[0], 2), 2**(-0.5)*traveldistance(x,x), 2*traveltendency(x,x)), fontsize='22'))
    (ax.set_ylabel('%s, $SD_y$=%s, $td_y$=%.2f, $tt_y$=%.2f'
                   %(name_indicator_2, round(dm[1], 2), 2**(-0.5)*traveldistance(y,y),2*traveltendency(y,y)), fontsize='22'))
###------------------------------------------Cosmetics----------------------------------------------------------
#limit of Axes
    ax.set_xlim([0.5,5+0.5])
    ax.set_ylim([0.5,5+0.5])
#add arrows
    line=plt.plot(x_reloc, y_reloc, markersize=0, color='cornflowerblue', zorder=-5)[0]
    add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),arrowstyle='->', arrowsize=4)
#start and end value
    start= ax.scatter(x_reloc[0],y_reloc[0],s=size[0], color='limegreen', label='first rating', zorder=45)
    end= ax.scatter(x_reloc[-1],y_reloc[-1],s=(size[-1]/10),color='gold', label='last rating', zorder=50)
#plot of meanvalue
    ev_list_x=np.zeros(len(x_data))
    ev_list_y=np.zeros(len(y_data))
    tt_list=np.zeros(len(x_data))
    for j in range(len(x_data)): #slope of rater
        ev_list_x[j]+= meanlist(x_data[j])
    for j in range(len(y_data)):
        ev_list_y[j]+= meanlist(y_data[j])
    const_for_ev_x=np.full(shape=50, fill_value=m[1])
    const_for_ev_y=np.full(shape=50, fill_value=m[0])
    #point of ev
    ax.scatter([m[0]],[m[1]],s=300, color='crimson',
                                label='EV: (%s$\pm$%s,%s$\pm$%s$)^T$'
                               %(round(m[0], 2), round(stdlist(ev_list_x)/np.sqrt(len(x_data)), 2),
                                 round(m[1], 2), round(stdlist(ev_list_y)/np.sqrt(len(y_data)), 2)),
               zorder=35)
    #errorbar
    #ev x error
    ax.plot(np.linspace(m[0]-stdlist(ev_list_x)/np.sqrt(len(x_data)),
                    m[0]+stdlist(ev_list_x)/np.sqrt(len(x_data)), 50),
            const_for_ev_x, 
            color='whitesmoke', linewidth=3, zorder=40)
    ax.plot(np.linspace(m[0]-stdlist(ev_list_x)/np.sqrt(len(x_data)),
                    m[0]+stdlist(ev_list_x)/np.sqrt(len(x_data)), 50),
            const_for_ev_x,
            color='crimson', linewidth=7, zorder=35)
    #ev y error
    ax.plot(const_for_ev_y,
            np.linspace(m[1]-stdlist(ev_list_y)/np.sqrt(len(y_data)),
                    m[1]+stdlist(ev_list_y)/np.sqrt(len(y_data)), 50),
            color='whitesmoke', linewidth=3, zorder=40)
    ax.plot(const_for_ev_y,
            np.linspace(m[1]-stdlist(ev_list_y)/np.sqrt(len(y_data)),
                    m[1]+stdlist(ev_list_y)/np.sqrt(len(y_data)), 50),
            color='crimson', linewidth=7, zorder=35)
#plot of deviation ellipse
    ax.plot(cx,cy, ls='-.',color='crimson',
                            label='ipd=%.2f'%(round(dens, 2)), zorder=30)
#title
    ax.set_title('%s, MSD=%.2f, SDD=%.2f'
                                 %(title, round(MSD, 2), round(SDD,2)), fontsize='21')
#plot of constant fits
    ax.plot(xfit_xconst,yfit_xconst, ls='-.', color='black', 
                                label='y =%.2f, $\chi_{y}^2$=%.2f'
                            %(poptx[0], chi_xconst(x,y, x_err, y_err)), zorder=20)
    ax.plot(xfit_yconst,yfit_yconst, ls='-.', color='black', 
                                label='x =%.2f, $\chi_{x}^2$=%.2f'
                            %(popty[0], chi_yconst(x,y, x_err, y_err)), zorder=20)
#plot of linear fits
    ax.plot(xfit,yfit,linewidth='3',color='fuchsia',
                                label='m =%.2f, $\chi_{lin}^2$=%.2f'
                            %(popt[0], chi_linear(x,y, x_err, y_err)), zorder=25)
#legend
    ax.legend(ncol=2, fontsize='19').set_zorder(100)
###--------------------------------------------Saving---------------------------------------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()

In [ ]:
#2D ASSG with slicing

def plot_2D_ASSG_with_slicing(start_time_slot, end_time_slot, path_data1, path_data2, path_fig, title, name_indicator_1, name_indicator_2):
    x_data_not_arranged=np.loadtxt(path_data1, unpack=True)
    x_data=np.array(x_data_not_arranged).T
    y_data_not_arranged=np.loadtxt(path_data2, unpack=True)
    y_data=np.array(y_data_not_arranged).T
    all_ratings_x=np.zeros(len(x_data_not_arranged[0])*len(x_data[1]))
    for j in range(len(x_data_not_arranged[0])):#slope of rater
        for k in range(len(x_data[0])):
            position_rating_x=k+j*len(x_data[j])
            all_ratings_x[position_rating_x]=+(x_data[j][k])
    all_ratings_y=np.zeros(len(y_data_not_arranged[1])*len(y_data[1]))
    for j in range(len(y_data_not_arranged[0])):#slope of rater
        for k in range(len(y_data[0])):
            position_rating_y=k+j*len(y_data[j])
            all_ratings_y[position_rating_y]=+(y_data[j][k])
    x_not_sliced, x_err_not_sliced = ratings_mean_and_error(all_ratings_x, len(x_data_not_arranged[1]), len(x_data[1]))
    y_not_sliced, y_err_not_sliced = ratings_mean_and_error(all_ratings_y, len(y_data_not_arranged[1]), len(y_data[1]))
    
    #slicing
    x, x_err = x_not_sliced[start_time_slot:end_time_slot], x_err_not_sliced[start_time_slot:end_time_slot]
    y, y_err = y_not_sliced[start_time_slot:end_time_slot], y_err_not_sliced[start_time_slot:end_time_slot]
    size=time(x,y)
    x_reloc, y_reloc = relocating(x,y)

###--------------------definations for the values---------------------------
#shorting of names for meanvalue and deviation
    m = [meanlist(x), meanlist(y)]
    dm = std(x,y)
#angle for deviation ellipse
    phi = np.linspace(0,2*np.pi,100)
#defination of deviation ellipse
    cx = m[0]+dm[0]*np.cos(phi)
    cy = m[1]+dm[1]*np.sin(phi)
#middle SD
    MSD = 0.5*(dm[0]+dm[1]) 
#SD difference
    SDD = np.absolute(dm[0]-dm[1])
#fitting
#linear
    popt, pcov = curve_fit(lin,x,y, bounds=([-5,-25], [5, 25]))
    xfit=np.linspace(0, 6)
    yfit=popt[0]*xfit+popt[1]
#xconst
    poptx, pcovx = curve_fit(xconst,x,y)
    xfit_xconst=np.linspace(0, 6)
    yfit_xconst=poptx[0]+0*xfit_xconst
#yconst
    popty, pcovy = curve_fit(xconst,y,x)
    yfit_yconst=np.linspace(0, 6)
    xfit_yconst=popty[0]+0*yfit_yconst
#inner point density
    dens=density(x,y) 
###---------------------------Plots-----------------------------------------
#size
    fig, ax = plt.subplots(figsize=(12,12))
#add grid
    grid_chessboard(5+1,5+1)
#points of ASSG-plot
    df = pd.DataFrame({'z': (np.linspace(0, len(x), len(x)))})
    dg = pd.DataFrame({'z': (np.linspace(len(x), 0, len(x)))})
    scatter_label= ax.scatter(x_reloc[0],y_reloc[0],s=size[0], c='none',label='td=%.2f, tt=%.2f' 
                                        %(traveldistance(x,y),traveltendency(x,y)), zorder=3)
    scatter= ax.scatter(x_reloc,y_reloc,s=size, c=df.z, cmap='Blues', zorder=4)
    scatter= ax.scatter(x_reloc,y_reloc,s=np.array(size)*1.2+100, c=dg.z, cmap='Blues', zorder=3)
#color errorbars
    #plt.errorbar(x_reloc, y_reloc, y_err, x_err, fmt='none', ecolor='whitesmoke', elinewidth=3, zorder=10)
    errorbar_color = (np.linspace(0, len(x), len(x)))
    cmap = cm.Blues
    norm = Normalize(vmin=errorbar_color.min(), vmax=errorbar_color.max())
    plt.errorbar(x_reloc, y_reloc, x_err, y_err, fmt='none', ecolor=cmap(norm(errorbar_color)), elinewidth=7, zorder=8)
    
    errorbar_color_inside = (np.linspace(len(x), 0, len(x)))
    cmap = cm.Blues
    norm = Normalize(vmin=errorbar_color_inside.min(), vmax=errorbar_color_inside.max())
    plt.errorbar(x_reloc, y_reloc, x_err, y_err, fmt='none', ecolor=cmap(norm(errorbar_color_inside)),
                 elinewidth=3, zorder=10)
    
    plt.errorbar(x_reloc[-1], y_reloc[-1], x_err[-1], y_err[-1], fmt='none',
                 ecolor=cmap(norm(errorbar_color))[-1],
                 label='$\overline{\Delta x}$=%.2f $\pm$ %.2f, \n$\overline{\Delta y}$=%.2f $\pm$ %.2f'
                 %(meanlist(x_err), stdlist(x_err), meanlist(y_err), stdlist(y_err)), zorder=1)
    
    points=np.array([x_reloc, y_reloc]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mc.LineCollection(segments, cmap='Blues', norm=norm)
    ax.add_collection(lc)
    lc.set_array(np.linspace(10, len(x), len(x)))
    
    
    
    
    (ax.set_xlabel('%s, $SD_x$=%s, $td_x$=%.2f, $tt_x$=%.2f'
                   %(name_indicator_1, round(dm[0], 2), 2**(-0.5)*traveldistance(x,x), 2*traveltendency(x,x)), fontsize='22'))
    (ax.set_ylabel('%s, $SD_y$=%s, $td_y$=%.2f, $tt_y$=%.2f'
                   %(name_indicator_2, round(dm[1], 2), 2**(-0.5)*traveldistance(y,y),2*traveltendency(y,y)), fontsize='22'))
###------------------------------------------Cosmetics----------------------------------------------------------
#limit of Axes
    ax.set_xlim([0.5,5+0.5])
    ax.set_ylim([0.5,5+0.5])
#add arrows
    line=plt.plot(x_reloc, y_reloc, markersize=0, color='cornflowerblue', zorder=-5)[0]
    add_arrow(ax, line, arrow_locs=np.linspace(0, 1, 30),arrowstyle='->', arrowsize=4)
#start and end value
    start= ax.scatter(x_reloc[0],y_reloc[0],s=size[0], color='limegreen', zorder=45)
    end= ax.scatter(x_reloc[-1],y_reloc[-1],s=(size[-1]/10),color='gold', zorder=50)
#plot of meanvalue
    ev_list_x=np.zeros(len(x_data))
    ev_list_y=np.zeros(len(y_data))
    tt_list=np.zeros(len(x_data))
    for j in range(len(x_data)): #slope of rater
        ev_list_x[j]+= meanlist(x_data[j])
    for j in range(len(y_data)):
        ev_list_y[j]+= meanlist(y_data[j])
    const_for_ev_x=np.full(shape=50, fill_value=m[1])
    const_for_ev_y=np.full(shape=50, fill_value=m[0])
    #point of ev
    ax.scatter([m[0]],[m[1]],s=300, color='crimson',
                                label='EV: (%s$\pm$%s,%s$\pm$%s$)^T$'
                               %(round(m[0], 2), round(stdlist(ev_list_x)/np.sqrt(len(x_data)), 2),
                                 round(m[1], 2), round(stdlist(ev_list_y)/np.sqrt(len(y_data)), 2)),
               zorder=35)
    #errorbar
    #ev x error
    ax.plot(np.linspace(m[0]-stdlist(ev_list_x)/np.sqrt(len(x_data)),
                    m[0]+stdlist(ev_list_x)/np.sqrt(len(x_data)), 50),
            const_for_ev_x, 
            color='whitesmoke', linewidth=3, zorder=40)
    ax.plot(np.linspace(m[0]-stdlist(ev_list_x)/np.sqrt(len(x_data)),
                    m[0]+stdlist(ev_list_x)/np.sqrt(len(x_data)), 50),
            const_for_ev_x,
            color='crimson', linewidth=7, zorder=35)
    #ev y error
    ax.plot(const_for_ev_y,
            np.linspace(m[1]-stdlist(ev_list_y)/np.sqrt(len(y_data)),
                    m[1]+stdlist(ev_list_y)/np.sqrt(len(y_data)), 50),
            color='whitesmoke', linewidth=3, zorder=40)
    ax.plot(const_for_ev_y,
            np.linspace(m[1]-stdlist(ev_list_y)/np.sqrt(len(y_data)),
                    m[1]+stdlist(ev_list_y)/np.sqrt(len(y_data)), 50),
            color='crimson', linewidth=7, zorder=35)
#plot of deviation ellipse
    ax.plot(cx,cy, ls='-.',color='crimson',
                            label='ipd=%.2f'%(round(dens, 2)), zorder=30)
#title
    ax.set_title('%s, MSD=%.2f, SDD=%.2f'
                                 %(title, round(MSD, 2), round(SDD,2)), fontsize='21')
#plot of constant fits
    ax.plot(xfit_xconst,yfit_xconst, ls='-.', color='black', 
                                label='y =%.2f, $\chi_{y}^2$=%.2f'
                            %(poptx[0], chi_xconst(x,y, x_err, y_err)), zorder=20)
    ax.plot(xfit_yconst,yfit_yconst, ls='-.', color='black', 
                                label='x =%.2f, $\chi_{x}^2$=%.2f'
                            %(popty[0], chi_yconst(x,y, x_err, y_err)), zorder=20)
#plot of linear fits
    ax.plot(xfit,yfit,linewidth='3',color='fuchsia',
                                label='m =%.2f, $\chi_{lin}^2$=%.2f'
                            %(popt[0], chi_linear(x,y, x_err, y_err)), zorder=25)
#legend
    ax.legend(ncol=2, fontsize='19').set_zorder(100)
###--------------------------------------------Saving---------------------------------------------------------
    plt.savefig(path_fig, bbox_inches='tight')
    plt.show()

In [ ]:
#arrows with add_arrow(line)
def add_arrow(
    axes, line, arrow_locs=[0.2, 0.4, 0.6, 0.8],
    arrowstyle='->', arrowsize=1, transform=None):
    if not isinstance(line, mlines.Line2D):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line.get_xdata(), line.get_ydata()

    arrow_kw = {
        "arrowstyle": arrowstyle,
        "mutation_scale": 10 * arrowsize,
    }

    color = 'cornflowerblue'
    #line.get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line.get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData
    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        #p= mpatches.FancyArrow(arrow_tail, arrow_head, transform=transform, color=color, width=0, 
        #                       head_width=5, head_length=5,length_includes_head=True, 
        #                       overhang=0.3, zorder=10)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

def add_arrow_hotpink(
    axes, line, arrow_locs=[0.2, 0.4, 0.6, 0.8],
    arrowstyle='->', arrowsize=1, transform=None):
    if not isinstance(line, mlines.Line2D):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line.get_xdata(), line.get_ydata()

    arrow_kw = {
        "arrowstyle": arrowstyle,
        "mutation_scale": 10 * arrowsize,
    }

    color = 'hotpink'
    #line.get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line.get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData
    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

def add_arrow_lightcoral(
    axes, line, arrow_locs=[0.2, 0.4, 0.6, 0.8],
    arrowstyle='->', arrowsize=1, transform=None):
    if not isinstance(line, mlines.Line2D):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line.get_xdata(), line.get_ydata()

    arrow_kw = {
        "arrowstyle": arrowstyle,
        "mutation_scale": 10 * arrowsize,
    }

    color = 'lightcoral'
    #line.get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line.get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData
    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

In [ ]:
### Chi square tests
#chisquare for linear function
def chi_linear(x, y, x_err_input, y_err_input):
    popt, pcov = curve_fit(lin,x,y)
    xfit=np.linspace(0, 6)
    yfit=popt[0]*xfit+popt[1]
    x_err=np.zeros(len(x_err_input))
    y_err=np.zeros(len(y_err_input))
    for i in range(len(x_err_input)):
        if x_err_input[i]==0:
            x_err[i]+=meanlist(x_err_input)/len(x_err_input)
        else:
            x_err[i]+=x_err_input[i]
    for i in range(len(y_err_input)):
        if y_err_input[i]==0:
            y_err[i]+=meanlist(y_err_input)/len(y_err_input)
        else:
            y_err[i]+=y_err_input[i]
    if np.sum((np.array(y)-lin(x,popt[0], popt[1]))**2+(np.array(x)-lin_x_value(y,popt[0], popt[1]))**2)==0:
        return 0
    else:
        quad = (((np.array(x)-(np.array(x)
                                +popt[0]*np.array(y)+popt[0]*popt[1])/(popt[0]**2+1))**2
                  +(np.array(y)-popt[0]*(np.array(x)+popt[0]*np.array(y)
                                         +popt[0]*popt[1])/(popt[0]**2+1)-popt[1])**2)**0.5/
                ((np.array(x_err)+np.array(y_err))**0.5))
        chi=np.sum(quad)/len(x)
    return chi
def lin_x_value(y, m, b):
    return (y-b)/m
#constat functions for curve_fit
def xconst(x, c):
    x=np.array(x)
    return c
#chisquare for xconst
def chi_xconst(x, y, x_err_input, y_err_input):
    poptx, pcovx = curve_fit(xconst,x,y)
    xfit_xconst=np.linspace(0, 6)
    yfit_xconst=poptx[0]+0*xfit_xconst
    y_err=np.zeros(len(y_err_input))
    for i in range(len(y_err_input)):
        if y_err_input[i]==0:
            y_err[i]+=meanlist(y_err_input)/len(y_err_input)
        else:
            y_err[i]+=y_err_input[i]
    if np.sum((np.array(y)-xconst(x,poptx[0]))**2)==0:
        return 0
    else:
        quad = (((np.array(y)-xconst(x,poptx[0]))**2)/(np.array(y_err)))
        chimin=np.sum(quad)/len(x)
        return chimin
#chisquare for yconst
def chi_yconst(x, y, x_err_input, y_err_input):
    popty, pcovy = curve_fit(xconst,y,x)
    yfit_yconst=np.linspace(0, 6)
    xfit_yconst=popty[0]+0*yfit_yconst
    x_err=np.zeros(len(x_err_input))
    for i in range(len(x_err_input)):
        if x_err_input[i]==0:
            x_err[i]+=meanlist(x_err_input)/len(x_err_input)
        else:
            x_err[i]+=x_err_input[i]
    if np.sum((np.array(x)-xconst(y,popty[0]))**2)==0:
        return 0
    else:
        quad = (((np.array(x)-xconst(y,popty[0]))**2)/(np.array(x_err)))
        chimin=np.sum(quad)/len(y)
        return chimin

#chisquare for linear function no errors
def chi_linear_no_errors(v1, v2):
    m = mean(v1,v2)
    popt, pcov = curve_fit(lin,v1,v2)
    xfit=np.linspace(0, 6)
    yfit=popt[0]*xfit+popt[1]
    if np.sum((np.array(v2)-lin(v1, popt[0], popt[1])**2))==0:
        return 0
    else:
        quad = ((((np.array(v2)-lin(v1,popt[0], popt[1]))/
                 (((v1-m[0])**2))**0.5) ** 2))
        chimin=np.sum(quad)/len(v1)
        return chimin
def chi_quad_no_errors(v1, v2):
    popt, pcov = curve_fit(quadfkt,v1,v2)
    xfit=np.linspace(0, 6)
    yfit=popt[1]*xfit**2+popt[0]*xfit+popt[2]
    if np.sum((np.array(v2)-quadfkt(v1,popt[1], popt[0], popt[2])**2))==0:
        return 0
    else:
        quad = ((((np.array(v2)-quadfkt(v1,popt[1], popt[0], popt[2]))/
                  (((v1-m[0])**2))**0.5) ** 2))
        chimin=np.sum(quad)/len(v1)
        return chimin
#chisquare for xconst
def chi_xconst_no_errors(v1, v2):
    m = mean(v1,v2)
    poptx, pcovx = curve_fit(xconst,v1,v2)
    xfit_xconst=np.linspace(0, 6)
    yfit_xconst=poptx[0]+0*xfit_xconst
    if np.sum((np.array(v2)-xconst(v1,poptx[0]))**2)==0:
        return 0
    else:
        quad = ((((np.array(v2)-xconst(v1,poptx[0]))/
                  (((v1-m[0])**2))**0.5) ** 2))
        chimin=np.sum(quad)/len(v1)
        return chimin
#chisquare for yconst
def chi_yconst_no_errors(v1, v2):
    m = mean(v1,v2)
    popty, pcovy = curve_fit(xconst,v2,v1)
    yfit_yconst=np.linspace(0, 6)
    xfit_yconst=popty[0]+0*yfit_yconst
    if np.sum((np.array(v2)-xconst(v1,popty[0]))**2)==0:
        return 0
    else:
        quad = (((np.array(v1)-xconst(v2,popty[0]))/(((v2-m[1])**2))**0.5) ** 2)
        chimin=np.sum(quad)/len(v2)
        return chimin